In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

# define function to load datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 36)
    return files, targets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load train, test, and validation datasets
data_files, data_targets = load_dataset('/content/drive/MyDrive/ASL_FingerSpelling_Dataset')
# load list of names
names = [item[17:19] for item in sorted(glob("/content/drive/MyDrive/ASL_FingerSpelling_Dataset/*/"))]

X_train, X_test, y_train, y_test = train_test_split(data_files, data_targets, test_size=0.2, random_state=42)

In [ ]:
# print statistics about the dataset
##fig = plt.Figure=(figsize=(20,5))
print('There are %s total images.\n' % len(np.hstack([X_train, X_test])))
print('There are %d training images.' % len(X_train))
print('There are %d total categories.' % len(names))
print('There are %d testing images.' % len(X_test))

There are 2515 total images.

There are 2012 training images.
There are 36 total categories.
There are 503 testing images.


In [ ]:
# Pre-process the Data
!pip install keras_preprocessing

from keras_preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_targets=y_train
test_targets=y_test
train_tensors = paths_to_tensor(X_train).astype('float32')/255
test_tensors = paths_to_tensor(X_test).astype('float32')/255

100%|██████████| 503/503 [00:03<00:00, 158.57it/s]


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
model2 = ResNet50(include_top=False,pooling= 'avg')
model2.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_2[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                        

In [ ]:
bottleneck_features_train_ResNet50 = np.asarray([model2.predict(np.expand_dims(tensor, axis=0), verbose=1)[0] for tensor in train_tensors],dtype=np.float32)

1/1 [==============================] - 0s 27ms/step


In [ ]:
bottleneck_features_test_ResNet50 = np.asarray([model2.predict(np.expand_dims(tensor, axis=0), verbose=1)[0] for tensor in test_tensors],dtype=np.float32)

1/1 [==============================] - 0s 26ms/step


In [ ]:
print(bottleneck_features_train_ResNet50.shape)
print(bottleneck_features_test_ResNet50.shape)

(2012, 2048)
(503, 2048)


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

from keras.layers import Activation, GlobalAveragePooling2D, Flatten, Dense
Model = Sequential()
Model.add(Dense(500, activation='relu',kernel_initializer='glorot_normal'))
Model.add(Dense(36, activation='softmax', kernel_initializer='glorot_normal'))
##Model.add(Dense(250, activation='relu',kernel_initializer='glorot_normal')) 
##Model.add(Dense(100, activation='relu',kernel_initializer='glorot_normal')) 
##Model.add(Dense(50, activation='relu',kernel_initializer='glorot_normal')) 

In [ ]:
##Model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])
Model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
##Model.compile(loss='categorical_crossentropy', optimizer="SGD", metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint

In [ ]:


checkpointer = ModelCheckpoint(filepath='/content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2', 
                               monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history2=Model.fit(bottleneck_features_train_ResNet50, train_targets, validation_split=0.1, epochs=250, batch_size=32, callbacks=[checkpointer], verbose=1)

Epoch 1/250
53/57 [==========================>...] - ETA: 0s - loss: 3.6249 - accuracy: 0.0448
Epoch 1: val_accuracy improved from -inf to 0.04950, saving model to /content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2


57/57 [==============================] - 1s 13ms/step - loss: 3.6179 - accuracy: 0.0470 - val_loss: 3.5697 - val_accuracy: 0.0495
Epoch 2/250
53/57 [==========================>...] - ETA: 0s - loss: 3.4207 - accuracy: 0.0890
Epoch 2: val_accuracy improved from 0.04950 to 0.09901, saving model to /content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2


57/57 [==============================] - 1s 10ms/step - loss: 3.4144 - accuracy: 0.0890 - val_loss: 3.2833 - val_accuracy: 0.0990
Epoch 3/250
48/57 [========================>.....] - ETA: 0s - loss: 3.2063 - accuracy: 0.1198
Epoch 3: val_accuracy improved from 0.09901 to 0.15842, saving model to /content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2


57/57 [==============================] - 1s 13ms/step - loss: 3.1929 - accuracy: 0.1215 - val_loss: 3.0576 - val_accuracy: 0.1584
Epoch 4/250
52/57 [==========================>...] - ETA: 0s - loss: 2.9832 - accuracy: 0.1677
Epoch 4: val_accuracy improved from 0.15842 to 0.18317, saving model to /content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2


57/57 [==============================] - 1s 16ms/step - loss: 2.9764 - accuracy: 0.1669 - val_loss: 2.8873 - val_accuracy: 0.1832
Epoch 5/250
48/57 [========================>.....] - ETA: 0s - loss: 2.8072 - accuracy: 0.2051
Epoch 5: val_accuracy improved from 0.18317 to 0.19307, saving model to /content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2


...

57/57 [==============================] - 1s 11ms/step - loss: 0.3852 - accuracy: 0.8669 - val_loss: 0.4643 - val_accuracy: 0.8515
Epoch 108/250
57/57 [==============================] - ETA: 0s - loss: 0.3726 - accuracy: 0.8602
Epoch 108: val_accuracy did not improve from 0.85149
57/57 [==============================] - 0s 4ms/step - loss: 0.3726 - accuracy: 0.8602 - val_loss: 0.4485 - val_accuracy: 0.8366
Epoch 109/250
51/57 [=========================>....] - ETA: 0s - loss: 0.3890 - accuracy: 0.8634
Epoch 109: val_accuracy did not improve from 0.85149
57/57 [==============================] - 0s 4ms/step - loss: 0.3812 - accuracy: 0.8657 - val_loss: 0.6030 - val_accuracy: 0.7475
Epoch 110/250
57/57 [==============================] - ETA: 0s - loss: 0.3631 - accuracy: 0.8685
Epoch 110: val_accuracy did not improve from 0.85149
57/57 [==============================] - 0s 4ms/step - loss: 0.3631 - accuracy: 0.8685 - val_loss: 0.6668 - val_accuracy: 0.7673
Epoch 111/250
53/57 [============

57/57 [==============================] - 1s 15ms/step - loss: 0.3373 - accuracy: 0.8867 - val_loss: 0.3809 - val_accuracy: 0.8762
Epoch 124/250
49/57 [========================>.....] - ETA: 0s - loss: 0.3255 - accuracy: 0.8833
Epoch 124: val_accuracy did not improve from 0.87624
57/57 [==============================] - 0s 6ms/step - loss: 0.3357 - accuracy: 0.8785 - val_loss: 0.5290 - val_accuracy: 0.7822
Epoch 125/250
49/57 [========================>.....] - ETA: 0s - loss: 0.3172 - accuracy: 0.8954
Epoch 125: val_accuracy did not improve from 0.87624
57/57 [==============================] - 0s 5ms/step - loss: 0.3242 - accuracy: 0.8901 - val_loss: 0.4734 - val_accuracy: 0.8366
Epoch 126/250
51/57 [=========================>....] - ETA: 0s - loss: 0.3077 - accuracy: 0.8885
Epoch 126: val_accuracy did not improve from 0.87624
57/57 [==============================] - 0s 4ms/step - loss: 0.3232 - accuracy: 0.8812 - val_loss: 0.7258 - val_accuracy: 0.7723
Epoch 127/250
55/57 [============

57/57 [==============================] - 1s 11ms/step - loss: 0.3178 - accuracy: 0.8867 - val_loss: 0.4153 - val_accuracy: 0.8911
Epoch 137/250
46/57 [=======================>......] - ETA: 0s - loss: 0.2994 - accuracy: 0.8899
Epoch 137: val_accuracy did not improve from 0.89109
57/57 [==============================] - 0s 5ms/step - loss: 0.2955 - accuracy: 0.8939 - val_loss: 0.5375 - val_accuracy: 0.8119
Epoch 138/250
55/57 [===========================>..] - ETA: 0s - loss: 0.3056 - accuracy: 0.8932
Epoch 138: val_accuracy did not improve from 0.89109
57/57 [==============================] - 0s 4ms/step - loss: 0.3068 - accuracy: 0.8928 - val_loss: 0.4165 - val_accuracy: 0.8465
Epoch 139/250
52/57 [==========================>...] - ETA: 0s - loss: 0.2904 - accuracy: 0.8960
Epoch 139: val_accuracy did not improve from 0.89109
57/57 [==============================] - 0s 4ms/step - loss: 0.2880 - accuracy: 0.8972 - val_loss: 0.5085 - val_accuracy: 0.8119
Epoch 140/250
53/57 [============

57/57 [==============================] - 1s 11ms/step - loss: 0.2167 - accuracy: 0.9182 - val_loss: 0.3027 - val_accuracy: 0.9010
Epoch 195/250
52/57 [==========================>...] - ETA: 0s - loss: 0.2037 - accuracy: 0.9213
Epoch 195: val_accuracy did not improve from 0.90099
57/57 [==============================] - 0s 4ms/step - loss: 0.2201 - accuracy: 0.9166 - val_loss: 0.3361 - val_accuracy: 0.8812
Epoch 196/250
55/57 [===========================>..] - ETA: 0s - loss: 0.1972 - accuracy: 0.9324
Epoch 196: val_accuracy did not improve from 0.90099
57/57 [==============================] - 0s 4ms/step - loss: 0.1965 - accuracy: 0.9331 - val_loss: 0.3850 - val_accuracy: 0.8960
Epoch 197/250
50/57 [=========================>....] - ETA: 0s - loss: 0.1948 - accuracy: 0.9319
Epoch 197: val_accuracy did not improve from 0.90099
57/57 [==============================] - 0s 4ms/step - loss: 0.2018 - accuracy: 0.9287 - val_loss: 0.3826 - val_accuracy: 0.8614
Epoch 198/250
53/57 [============

57/57 [==============================] - 1s 16ms/step - loss: 0.1829 - accuracy: 0.9337 - val_loss: 0.2751 - val_accuracy: 0.9109
Epoch 225/250
46/57 [=======================>......] - ETA: 0s - loss: 0.1683 - accuracy: 0.9429
Epoch 225: val_accuracy did not improve from 0.91089
57/57 [==============================] - 0s 6ms/step - loss: 0.1767 - accuracy: 0.9425 - val_loss: 0.3748 - val_accuracy: 0.8960
Epoch 226/250
53/57 [==========================>...] - ETA: 0s - loss: 0.1728 - accuracy: 0.9304
Epoch 226: val_accuracy did not improve from 0.91089
57/57 [==============================] - 0s 5ms/step - loss: 0.1751 - accuracy: 0.9309 - val_loss: 0.3118 - val_accuracy: 0.8812
Epoch 227/250
56/57 [============================>.] - ETA: 0s - loss: 0.1827 - accuracy: 0.9381
Epoch 227: val_accuracy did not improve from 0.91089
57/57 [==============================] - 0s 5ms/step - loss: 0.1820 - accuracy: 0.9381 - val_loss: 0.2621 - val_accuracy: 0.9109
Epoch 228/250
56/57 [============

In [ ]:
Model2 = Sequential()
Model2.add(Dense(800, activation='relu',kernel_initializer='glorot_normal'))
Model2.add(Dense(36, activation='softmax', kernel_initializer='glorot_normal'))
Model2.load_weights('/content/saved_models/weights.best.F.hdf5_TransferLearning_ResNet50_2')
import time
# Start timing
start_time = time.time()
ResNet50_predictions = [Model2.predict(np.expand_dims(tensor, axis=0))[0] for tensor in bottleneck_features_test_ResNet50 ]
# End timing
end_time = time.time()
print("Time Taken: {} seconds".format(end_time-start_time))

1/1 [==============================] - 0s 18ms/step
Time Taken: 31.787902355194092 seconds


In [ ]:
## report test accuracy
predicted_classes = np.argmax(np.round(ResNet50_predictions),axis=1)
test_classes = np.argmax(np.round(test_targets),axis=1)
from sklearn.metrics import classification_report
print(classification_report(test_classes, predicted_classes))
from sklearn.metrics import confusion_matrix
confusion_mtx = confusion_matrix(test_classes, predicted_classes)
print(confusion_mtx)


              precision    recall  f1-score   support

           0       0.53      1.00      0.70        16
           1       0.83      0.91      0.87        11
           2       0.83      0.83      0.83        12
           3       1.00      1.00      1.00        12
           4       1.00      0.92      0.96        13
           5       1.00      1.00      1.00        17
           6       1.00      0.67      0.80        18
           7       1.00      0.92      0.96        13
           8       1.00      1.00      1.00        16
           9       1.00      1.00      1.00        17
          10       0.90      1.00      0.95        19
          11       1.00      1.00      1.00        12
          12       1.00      1.00      1.00        11
          13       1.00      0.95      0.97        20
          14       1.00      1.00      1.00        19
          15       1.00      1.00      1.00        11
          16       0.90      0.90      0.90        21
          17       0.83    